<a href="https://colab.research.google.com/github/ReetaMercy/HexSoftwares_Sentimental_Analysis/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('movie_review.csv')
df.head()
df.shape

(64720, 6)

In [5]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
corpus = []
for i in range(0, 64720):
  text = re.sub('[^a-zA-Z]', ' ', df['text'][i])
  text = text.lower()
  text = text.split()
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus.append(text)

In [7]:
corpus

['film adapt comic book plenti success whether superhero batman superman spawn gear toward kid casper arthous crowd ghost world never realli comic book like hell',
 'starter creat alan moor eddi campbel brought medium whole new level mid part seri call watchmen',
 'say moor campbel thoroughli research subject jack ripper would like say michael jackson start look littl odd',
 'book graphic novel page long includ nearli consist noth footnot',
 'word dismiss film sourc',
 'get past whole comic book thing might find anoth stumbl block hell director albert allen hugh',
 'get hugh brother direct seem almost ludicr cast carrot top well anyth riddl better direct film set ghetto featur realli violent street crime mad genius behind menac ii societi',
 'ghetto question cours whitechapel london east end',
 'filthi sooti place whore call unfortun start get littl nervou mysteri psychopath carv profess surgic precis',
 'first stiff turn copper peter godley robbi coltran world not enough call inspecto

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [9]:
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, -1].values

In [13]:
import pickle
bow_path = 'movie_review.csv-bow_sentimental-model.pkl'
pickle.dump(cv, open(bow_path, "wb"))

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [15]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [16]:
import joblib
filename = 'movie_review.csv-sentimental-model.pkl'
joblib.dump(classifier, filename)

['movie_review.csv-sentimental-model.pkl']

In [17]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[4520 1970]
 [3062 3392]]


0.6112484548825711

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile = 'movie_review.csv-bow_sentimental-model.pkl'
cv = pickle.load(open(cvFile, "rb"))

In [20]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(64720, 1420)

In [21]:
import joblib
classifier = joblib.load('movie_review.csv-sentimental-model.pkl')

In [22]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

['neg' 'pos' 'pos' ... 'neg' 'neg' 'neg']


In [23]:
df['predicted_label'] = y_pred.tolist()
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag,predicted_label
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos,neg
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos,pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos,pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos,neg


In [25]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1jprl9pTbFFbvzV3Bj4pSDoXsNU-MHUdRukmTfNxArcc#gid=0


In [24]:
df.to_csv('movie_review.csv', sep = '\t', encoding = 'UTF-8', index = False)